In [ ]:

# Import necessary libraries
import os, mne, re
from mne.channels import make_standard_montage
from glob import glob
from mne_bids import BIDSPath, write_raw_bids, read_raw_bids
from pyprep.find_noisy_channels import NoisyChannels


# Set the folder for recreate the data in bids format downsampled 
root_bids = 'C:/Users/mfbpe/Desktop/DATA/2023_Eva_Freedom/bids/'

# Set the derivatives directory path
root_derivatives = root_bids.split('/bids')[0] + '/derivatives'

# Set the image directory path
root_image = 'C:/Users/mfbpe/Desktop/DATA/2023_Eva_Freedom/images_prep_eeg/'

# Change the current working directory to the derivatives directory
os.chdir(root_derivatives)

# Get a list of all file paths with .bdf extension in the source directory
all_files_path = sorted(glob(f'{root_bids}/*/*/*_eeg.vhdr*'), key=len)

# Define a dictionary with participant numbers as keys and a list of bad channels as values + a list of bad ica components

prtp_chs_out = {}
prtp_ica_out = {}

for part in all_files_path:
    # Extract the participant number from the file path
    n_part = part.split("\\")[1].split("-")[1]

    bidspath = BIDSPath(subject=n_part, task='constraint', datatype='eeg', root=root_bids)
    raw = read_raw_bids(bids_path=bidspath).load_data()

    # Apply a bandpass filter to the raw data 
    raw.filter(l_freq=.01, h_freq=40, n_jobs=-1)


    raw_copy = raw.copy().filter(1, None, n_jobs=-1) #to improve detection channels and ICA
    # Interpolate bad channels for participants in the prtp_chs_out list
    if n_part in list(prtp_chs_out):
        raw.info['bads'] = prtp_chs_out[n_part]
        raw = raw.interpolate_bads()
        raw_copy = raw_copy.interpolate_bads()
    else :
        noisy= NoisyChannels(raw_copy)
        noisy.find_all_bads()
        bad_chs = noisy.get_bads()
        raw.info['bads'] = bad_chs
        raw = raw.interpolate_bads()
        raw_copy = raw_copy.interpolate_bads()
        prtp_chs_out[n_part]=bad_chs

    # Perform Independent Component Analysis (ICA) on the copied data
    ica = mne.preprocessing.ICA(random_state=21, max_iter='auto')
    ica.fit(raw_copy)


    if n_part in list(prtp_ica_out):
        ica.exclude = prtp_ica_out[n_part]
        ica.apply(raw)
    fig = ica.plot_components(picks=np.arange(ica.n_components_), show = False) #create figures of the components with the one removed
    fig.savefig(f"{root_image}sub-{n_part}_ICA.png")

    del ica

    # Save the preprocessed raw data
    raw.save(f"sub-{n_part}_constraint_raw.fif", overwrite=True)

    del raw, raw_copy

# create the txt file

with open('list_bad_channels.txt', 'w') as f:
    f.write('dict = ' + repr(prtp_chs_out) + '\n')

with open('list_bad_ica.txt', 'w') as f:
    f.write('dict = ' + repr(prtp_ica_out) + '\n')    
